In [61]:
library(plyr, quietly = TRUE)
library(tidyverse,  quietly = TRUE)
library(tidyjson,  quietly = TRUE)
library(readxl)

In [2]:
# get name genes number  source

In [62]:
genesets_colnames = c("Source", "GeneSetID", "GeneSetName", "GeneList", "n_Genes")
genesets_MAT = matrix(nrow = 0, ncol = length(genesets_colnames))
colnames(genesets_MAT) = genesets_colnames
genesets_DF = as_tibble(genesets_MAT) %>%
  mutate(across(everything(), as.character))
genesets_DF$n_Genes = as.integer(genesets_DF$n_Genes)

In [63]:
genesets_empty_DF = genesets_DF

### MSigDB

In [64]:
msigdb_path = "data/human_geneSets/"

In [65]:
# read in the MSigDB .json file
MSigDB_hallmark = read_json("data/h.all.v2023.1.Hs.json", format = "json")[[2]][[1]]

In [66]:
MSigDB_geneSets = names(MSigDB_hallmark)

In [67]:
for (MSigDB_geneSet in MSigDB_geneSets){
    MSigDB_geneSet_genes = MSigDB_hallmark[[MSigDB_geneSet]]$geneSymbols %>% unlist()
    MSigDB_geneSet_n_Genes = length(MSigDB_geneSet_genes)
    genesets_DF = genesets_DF %>% 
    add_row(tibble_row(Source = "MSigDB",
            GeneSetID = MSigDB_geneSet,           
            GeneSetName = MSigDB_geneSet, 
            GeneList = str_c(MSigDB_geneSet_genes,  collapse = " "),
            n_Genes = MSigDB_geneSet_n_Genes))
    }

In [68]:
genesets_DF %>% subset(n_Genes < 200) %>% nrow()

[1] 27

In [69]:
genesets_DF %>% subset(n_Genes <= 100) %>% nrow()

[1] 13

In [70]:
dim(genesets_DF)

[1] 50  5

In [71]:
genesets_DF$n_Genes %>% max()

[1] 200

### NeST

In [72]:
NeST_systems_Orig = read_delim("data/NeST_table.txt", delim = "\t") %>% pull(`NEST ID`)

Rows: 50 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): NEST ID, name_new, Genes

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [73]:
NeST_All_DF = read_delim(file = "data/NeST_table_All.csv", delim = "," )

Rows: 131 Columns: 24
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (12): All_Genes, name, name_new, name_position, NEST ID, Note, shared na...
dbl (11): Camptothecin, CD437, Cisplatin, Etoposide, Gemcitabine, name_show,...
lgl  (1): selected

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [74]:
NeST_Orig_DF = NeST_All_DF %>%
subset(`NEST ID` %in% NeST_systems_Orig) %>%
subset(Size_All <= 200)

In [75]:
setdiff(NeST_systems_Orig, NeST_Orig_DF$`NEST ID`)

[1] "NEST:3"

In [76]:
n_toSample = 50 - nrow(NeST_Orig_DF)

In [77]:
NeST_added_DF = NeST_All_DF %>%
subset(Size_All <= 200) %>%
subset(str_detect(name_new, "NEST", negate = TRUE)) %>%
slice_sample(n = n_toSample) 

In [78]:
NeST_toUse_DF = rbind(NeST_Orig_DF, NeST_added_DF) %>%
select(c("NEST ID", "name_new", "All_Genes", "Size_All"))  %>%
rowwise() %>%
mutate(Source  = "NeST", 
       All_Genes = str_replace_all(string = All_Genes, pattern  = ",", replacement =  " ")) %>%
rename(GeneSetID = `NEST ID`,
       GeneSetName = name_new, 
       GeneList = All_Genes, 
       n_Genes = Size_All)

In [79]:
genesets_DF = genesets_DF %>% rbind(NeST_toUse_DF)

In [80]:
dim(genesets_DF)

[1] 100   5

### Gene expression

In [81]:
# https://doi.org/10.1016/j.cell.2022.05.013

In [82]:
geneExpression_DF = read_xlsx(path = "data/1-s2.0-S0092867422005979-mmc3.xlsx", sheet = "gene expression clusters")

New names:
• `` -> `...1`


In [83]:
geneExpression_DF = read_xlsx(path = "data/1-s2.0-S0092867422005979-mmc3.xlsx", sheet = "gene expression clusters") %>%
subset(!is.na(manual_annotation))  %>%
select(c("members", "manual_annotation")) %>%
rowwise() %>%
mutate(Source  = "Gene Expression", 
       members = str_replace_all(string = members, pattern  = ",", replacement =  " "),
       n_Genes = length(str_split(string = members, pattern = " ")[[1]])) %>%
rename(GeneSetName = manual_annotation, 
       GeneList = members) %>%
mutate(GeneSetID =  GeneSetName) %>%
subset(n_Genes <= 200)


New names:
• `` -> `...1`


In [84]:
dim(genesets_DF)

[1] 100   5

In [85]:
genesets_DF = genesets_DF %>% rbind(geneExpression_DF)

In [86]:
dim(genesets_DF)

[1] 125   5

### Perturbation

In [87]:
perturbation_DF = read_xlsx(path = "data/1-s2.0-S0092867422005979-mmc3.xlsx", sheet = "perturbation clusters") %>%
subset(!is.na(manual_annotation))  %>%
select(c("members", "manual_annotation")) %>%
rowwise() %>%
mutate(Source  = "Perturbation", 
       members = str_replace_all(string = members, pattern  = ",", replacement =  " "),
       n_Genes = length(str_split(string = members, pattern = " ")[[1]])) %>%
rename(GeneSetName = manual_annotation, 
       GeneList = members)%>%
mutate(GeneSetID =  GeneSetName) %>%
subset(n_Genes <= 200)

New names:
• `` -> `...1`


In [88]:
dim(perturbation_DF)

[1] 48  5

In [89]:
genesets_DF = genesets_DF %>% rbind(perturbation_DF)

In [90]:
dim(genesets_DF)

[1] 173   5

In [91]:
250 - 173 

[1] 77

### Disease

In [33]:
library(disgenet2r)
library(getPass)


Attaching package: ‘disgenet2r’


The following object is masked from ‘package:tidyr’:

    extract




In [34]:
pass = getPass::getPass("Enter the password: ")

Enter the password:  ········


In [35]:
disgenet_api_key <- get_disgenet_api_key(
                  email = "salkhairy@ucsd.edu", 
                  password = pass )

In [36]:
Sys.setenv(DISGENET_API_KEY= disgenet_api_key)

In [37]:
diseaseId_All_DF = read_delim("data//disease_associations.tsv", delim = "\t")


Rows: 30170 Columns: 7
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): diseaseId, diseaseName, diseaseType, diseaseClass, diseaseSemanticType
dbl (2): NofGenes, NofPmids

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [38]:
diseaseId_All_list = diseaseId_All_DF$diseaseName
names(diseaseId_All_list) = diseaseId_All_DF$diseaseId

In [39]:
diseaseId_All_list[1:4]

C0000727            C0000729            C0000731            C0000734 
   "Abdomen, Acute"  "Abdominal Cramps" "Abdomen distended"    "Abdominal mass"

In [40]:
# Read disease list
diseaseId_All_list = diseaseId_All_DF %>%
subset(diseaseType == "disease") %>%
subset(between(x = NofGenes, left = 5, right = 200)) %>%
subset(NofPmids > 1) %>%
pull(diseaseId)

In [53]:
diseaseId_All_DF %>% subset(diseaseId %in% diseaseId_All_list) %>% pull("NofGenes") %>% min()

[1] 5

In [54]:
# Sample 
diseaseId_list = sample(x = diseaseId_All_list, size = 500, replace = FALSE) # not all of them are successful in getting gene list

In [55]:
disease_DF = genesets_empty_DF
an.error.occured <- FALSE


In [56]:
for (diseaseId_ind in c(1:length(diseaseId_list))){
    print(diseaseId_ind)
    diseaseId = diseaseId_list[diseaseId_ind]
    diseaseName = diseaseId_All_list[diseaseId]
    
    tryCatch( { dis_res <- disease2gene( diseaseId_list[diseaseId_ind], database = "CURATED" ) }
          , error = function(e) {an.error.occured <<- TRUE} )
    
    if (class(dis_res) == 'character'){next}
        
    geneList = dis_res@qresult %>% pull(gene_symbol)
    
    disease_DF = disease_DF %>% 
    add_row(tibble_row(Source = "Disease",
            GeneSetID = diseaseId,
            GeneSetName = diseaseName, 
            GeneList = str_c(geneList,  collapse = " "),
            n_Genes = length(geneList)))
    }

[1] 1
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"detail\":\"The query didn't return any results\",\"status_code\":404}"
[1] "no results for the query"
[1] 2
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"detail\":\"The query didn't return any results\",\"status_code\":404}"
[1] "no results for the query"
[1] 3
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



...

[1] "{\"detail\":\"The query didn't return any results\",\"status_code\":404}"
[1] "no results for the query"
[1] 493
[1] 494
[1] 495
[1] 496
[1] 497
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"detail\":\"The query didn't return any results\",\"status_code\":404}"
[1] "no results for the query"
[1] 498
[1] 499
[1] 500
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"detail\":\"The query didn't return any results\",\"status_code\":404}"
[1] "no results for the query"


In [59]:
disease_toUse_DF = disease_DF %>% subset(n_Genes >=2) %>% head(n = 77)

In [60]:
dim(disease_toUse_DF)

[1] 77  5

In [92]:
genesets_DF = genesets_DF %>% rbind(disease_toUse_DF)

In [95]:
dim(disease_toUse_DF)

[1] 77  5

In [94]:
dim(genesets_DF)

[1] 250   5

In [96]:
write_delim(x = genesets_DF,file = "data/omics.txt" , delim = "\t")